# Project 2

## Problem Statement


The Housing & Development Board (HDB) flats are a type of public housing in Singapore. They are highly sought after by both Singaporean citizens and permanent residents due to their affordability and various amenities provided by the government.
Real estate agents play a crucial role in facilitating the buying and selling of properties, including HDB flats. Their expertise and knowledge of the local market are essential in helping clients make informed decisions and achieve the best possible outcomes. To effectively serve their clients and maximize their business potential, real estate agents need accurate and reliable price predictions for HDB flats.
To address this need, our team of data scientists has been engaged by a group of real estate agents who are planning to establish their own real estate agency. They have specifically identified HDB flats as their primary focus and want us to develop a price prediction model tailored to this housing type. The aim of our data science project is to create a robust and accurate price prediction model for HDB flats. The successful implementation of this price prediction model will enable the real estate agents to make data-driven decisions, offer competitive pricing recommendations, attract more clients, and establish themselves as trusted advisors in the HDB market.

## Aim of Project

- To create a working linear regression model to predict HDB prices
- To understand what are the key predictors of HDB prices
- Success of the model will be evaluated by its R-squared and RMSE

## Import Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score

%config InlineBackend.figure_format = 'retina'

## Import Datasets

In [3]:
housing_train = pd.read_csv('../datasets/train.csv')
housing_test = pd.read_csv('../datasets/test.csv')

/var/folders/r4/61_m3nz133bcpt1gp9t0vyq00000gn/T/ipykernel_1000/1963004826.py:1: DtypeWarning: Columns (42) have mixed types. Specify dtype option on import or set low_memory=False.
  housing_train = pd.read_csv('../datasets/train.csv')
/var/folders/r4/61_m3nz133bcpt1gp9t0vyq00000gn/T/ipykernel_1000/1963004826.py:2: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  housing_test = pd.read_csv('../datasets/test.csv')


In [5]:
housing_train.describe()

,id,floor_area_sqm,lease_commence_date,resale_price,Tranc_Year,Tranc_Month,mid_storey,lower,upper,mid,...,pri_sch_nearest_distance,vacancy,pri_sch_affiliation,pri_sch_latitude,pri_sch_longitude,sec_sch_nearest_dist,cutoff_point,affiliation,sec_sch_latitude,sec_sch_longitude
count,150634.000000,150634.00000,150634.000000,1.506340e+05,150634.000000,150634.000000,150634.000000,150634.000000,150634.000000,150634.000000,...,150634.000000,150634.000000,150634.000000,150634.000000,150634.000000,150634.000000,150634.000000,150634.000000,150634.000000,150634.000000
mean,93047.887496,97.19763,1992.448464,4.491615e+05,2016.489551,6.580500,8.284823,7.248370,9.321275,8.284823,...,395.172974,55.368190,0.113195,1.366268,103.839106,508.533220,210.146428,0.031879,1.365986,103.839340
std,53679.906247,24.40304,12.138829,1.433076e+05,2.752396,3.345468,5.508074,5.515155,5.507364,5.508074,...,234.760931,17.903918,0.316832,0.042669,0.072637,309.066265,20.010489,0.175677,0.042599,0.072630
min,1.000000,31.00000,1966.000000,1.500000e+05,2012.000000,1.000000,2.000000,1.000000,3.000000,2.000000,...,45.668324,20.000000,0.000000,1.274962,103.687724,38.913475,188.000000,0.000000,1.276029,103.687207
25%,46469.250000,75.00000,1984.000000,3.470000e+05,2014.000000,4.000000,5.000000,4.000000,6.000000,5.000000,...,227.083163,44.000000,0.000000,1.337289,103.773754,290.285883,188.000000,0.000000,1.337545,103.776008
50%,93077.500000,95.00000,1992.000000,4.200000e+05,2017.000000,7.000000,8.000000,7.000000,9.000000,8.000000,...,348.876691,54.000000,0.000000,1.364639,103.844210,447.377670,208.000000,0.000000,1.366132,103.842719
75%,139574.750000,112.00000,2001.000000,5.200000e+05,2019.000000,9.000000,11.000000,10.000000,12.000000,11.000000,...,502.179385,67.000000,0.000000,1.393910,103.898773,644.284099,224.000000,0.000000,1.391781,103.899872
max,185966.000000,280.00000,2019.000000,1.258000e+06,2021.000000,12.000000,50.000000,49.000000,51.000000,50.000000,...,3305.841039,110.000000,1.000000,1.456667,103.962919,3638.977233,260.000000,1.000000,1.453280,103.961105


In [3]:
# clean column names
housing_train.columns=housing_train.columns.str.lower()
housing_test.columns=housing_test.columns.str.lower()

In [4]:
# find all columns with NaN (train)
nan_cols_train=housing_train.columns[housing_train.isna().any()].tolist()
# find number of rows with NaN for each column in nan_cols
for i in nan_cols_train:
    print(f'no. of NaN for {i}: {housing_train[i].isna().sum()}')

print('\n')

# find all columns with NaN (test)
nan_cols_test=housing_test.columns[housing_test.isna().any()].tolist()
for i in nan_cols_test:
    print(f'no. of NaN for {i}: {housing_test[i].isna().sum()}')

no. of NaN for mall_nearest_distance: 829
no. of NaN for mall_within_500m: 92789
no. of NaN for mall_within_1km: 25426
no. of NaN for mall_within_2km: 1940
no. of NaN for hawker_within_500m: 97390
no. of NaN for hawker_within_1km: 60868
no. of NaN for hawker_within_2km: 29202


no. of NaN for mall_nearest_distance: 84
no. of NaN for mall_within_500m: 10292
no. of NaN for mall_within_1km: 2786
no. of NaN for mall_within_2km: 213
no. of NaN for hawker_within_500m: 10755
no. of NaN for hawker_within_1km: 6729
no. of NaN for hawker_within_2km: 3254


Note here that number of NaN for malls / hawkers (within a certain radius) decreases as radius increases, which makes sense if we assume that NaN stands for 0 malls / hawkers in the specified radius.

## Train

In [5]:
# drop dups
df_drop_dups=housing_train.drop_duplicates(subset=['longitude','latitude'])
df_w_na_no_dups=df_drop_dups[df_drop_dups['mall_nearest_distance'].isna()]

# index of rows with NaN in 'mall_nearest_distance'
idx_w_na=list(housing_train[housing_train['mall_nearest_distance'].isna()].index.values)
idx_w_na_no_dups=list(df_w_na_no_dups.index.values)
len(idx_w_na)

829

In [6]:
# find all rows with no NaN in 'mall_nearest_distance' and no dups
df_no_na_no_dups=df_drop_dups[df_drop_dups['mall_nearest_distance'].notna()]
idx_no_na_no_dups=list(df_no_na_no_dups.index.values)

In [7]:
# Defining a function 'shortest_l' that finds the row in df that has the shortest distance between the 'long_org' and 'lat_org' input,
# and returns the 'mall_nearest_distance' for that row
# find shortest length between 2 points
def shortest_l(idx,df,long_org,lat_org):
    dist=999
    new_info=[0,0,0]
    for i in idx:
        new_dist=((long_org - df.loc[i]['longitude'])**2 + (lat_org - df.loc[i]['latitude'])**2)**0.5
        if new_dist<dist:
            dist=new_dist
            new_info[0]=df.loc[i]['longitude']
            new_info[1]=df.loc[i]['latitude']
            new_info[2]=df.loc[i]['mall_nearest_distance']
    return new_info

In [8]:
idx=idx_no_na_no_dups
df=df_no_na_no_dups
# Looping through the housing_train dataframe to impute 'mall_nearest_distance' for rows with missing values
for i in idx_w_na: 
    housing_train.iloc[i,46]=shortest_l(idx,df,housing_train['longitude'][i],housing_train['latitude'][i])[2]

In [9]:
# replace NaN with 0 for cols with NaN
for i in nan_cols_train:
    housing_train[i] = housing_train[i].fillna(0)

In [10]:
# check for cols with NaN
def null_check(df):
    is_null = df.isna().sum().sort_values()
    is_null_df = is_null.to_frame()
    is_null_df = is_null_df.rename({0:'number of NaNs'}, axis = 1)
    is_null_df['% of total rows'] = is_null_df['number of NaNs']/df.shape[0]*100
    is_null_df = is_null_df[is_null_df.loc[:]!=0].dropna()
    return is_null_df

null_check(housing_train)

,number of NaNs,% of total rows


## Test

In [11]:
# drop dups
df_drop_dups_test=housing_test.drop_duplicates(subset=['longitude','latitude'])
df_w_na_no_dups_test=df_drop_dups_test[df_drop_dups_test['mall_nearest_distance'].isna()]

# index of rows with NaN in 'mall_nearest_distance'
idx_w_na_test=list(housing_test[housing_test['mall_nearest_distance'].isna()].index.values)
idx_w_na_no_dups=list(df_w_na_no_dups_test.index.values)
len(idx_w_na)

829

In [12]:
# find all rows with no NaN in 'mall_nearest_distance' and no dups
df_no_na_no_dups_test=df_drop_dups_test[df_drop_dups_test['mall_nearest_distance'].notna()]
idx_no_na_no_dups_test=list(df_no_na_no_dups_test.index.values)

In [13]:
idx=idx_no_na_no_dups_test
df=df_no_na_no_dups_test
# Looping through the housing_test dataframe to impute 'mall_nearest_distance' for rows with missing values
for i in idx_w_na_test: 
    housing_test.iloc[i,45]=shortest_l(idx,df,housing_test['longitude'][i],housing_test['latitude'][i])[2]

In [14]:
# replace NaN with 0 for cols with NaN
for i in nan_cols_test:
    housing_test[i] = housing_test[i].fillna(0)

In [15]:
# check for cols with NaN
def null_check(df):
    is_null = df.isna().sum().sort_values()
    is_null_df = is_null.to_frame()
    is_null_df = is_null_df.rename({0:'number of NaNs'}, axis = 1)
    is_null_df['% of total rows'] = is_null_df['number of NaNs']/df.shape[0]*100
    is_null_df = is_null_df[is_null_df.loc[:]!=0].dropna()
    return is_null_df

null_check(housing_test)

,number of NaNs,% of total rows


## Save

In [16]:
housing_train.to_csv('../datasets/train_pop_NaN.csv',index=False)
housing_test.to_csv('../datasets/test_pop_NaN.csv',index=False)